In [ ]:
import json
import sklearn
import numpy
import random
import scipy
import sklearn.datasets
import sklearn.feature_extraction.text
import sklearn.linear_model
import sklearn.naive_bayes
import sklearn.metrics
import sklearn.utils
import string
import io

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import StanfordNERTagger
import matplotlib.pyplot as plt

import os
java_path = "C:/Program Files/Java/jdk1.8.0_131/bin/java.exe"
os.environ['JAVAHOME'] = java_path

In [ ]:
with open('articles_with_texts_final.json') as data_file:
    articles = json.load(data_file)

In [ ]:
article_keys = articles[u'data'].keys()
vectorizer = \
sklearn.feature_extraction.text.TfidfVectorizer(sublinear_tf=True, 
                                                max_df=0.5, 
                                                stop_words='english')
_ = vectorizer.fit(articles[u'data'][k][u'text'] for k in article_keys[:4000])

In [ ]:
T_keys = [k for k in article_keys if (articles[u'data'][k]['most_emailed'] or articles[u'data'][k]['most_shared'] or \
                articles[u'data'][k]['most_viewed'])]
F_keys = [k for k in article_keys if not (articles[u'data'][k]['most_emailed'] or articles[u'data'][k]['most_shared'] or \
                articles[u'data'][k]['most_viewed'])]

In [273]:
#def feature1(c_articles) : #c_articles is [articles[u'data'][k] for k in keys]

#author_vectorizer = sklearn.feature_extraction.text.CountVectorizer()
#_ = author_vectorizer.fit(string.replace(string.replace(ca[u'byline'][u'original'], ' ', ''), '.', '') if ca[u'byline'] != [] else "" for ca in [articles[u'data'][k] for k in article_keys])

def author(cas):
    return author_vectorizer.transform(string.replace(string.replace(ca[u'byline'][u'original'], ' ', ''), '.', '') if ca[u'byline'] != [] else "" for ca in cas)
    
# truth with noise
def truth(cas):
    if len(cas) == 4000 :
        return numpy.reshape([(1 if random.randint(1,4) > 1 else 0) if k in T_keys else (1 if random.randint(1,4) == 1 else 0) for k in article_keys[:4000]], (-1, 1))
    else :
        return numpy.reshape([(1 if random.randint(1,4) > 1 else 0) if k in T_keys else (1 if random.randint(1,4) == 1 else 0) for k in article_keys[4000:]], (-1, 1))

st = StanfordNERTagger('C:\stanford-ner\classifiers\english.all.3class.distsim.crf.ser.gz', 'C:\stanford-ner\stanford-ner.jar', encoding='utf-8')
    
NE_count_file = 'NE_counts.txt'
    
def NE_count(text):
    tokenized_text = word_tokenize(text)
    t = st.tag(tokenized_text)
    n_persons = 0.0
    n_org = 0.0
    n_loc = 0.0
    if len(t) == 0 :
        return [0.0, 0.0, 0.0]
    (a, b) = t[0]
    if b == u'PERSON' :
        n_persons += 1
    elif b == u'LOCATION' :
        n_loc += 1
    elif b == u'ORGANIZATION' :
        n_org += 1
    prev = b
    for (a, b) in t[1:] :
        if b != prev :
            prev = b
            if b == u'PERSON' :
                n_persons += 1
            elif b == u'LOCATION' :
                n_loc += 1
            elif b == u'ORGANIZATION' :
                n_org += 1
    return [n_persons/len(tokenized_text), n_loc/len(tokenized_text), n_org/len(tokenized_text)]

def named_entity(keys):
    NE_count_dict = {}
    if NE_count_file != None :
        with open(NE_count_file, 'r') as f:
            for line in f:
                l = line.split()
                NE_count_dict[encode(l[0])] = [float(l[1]), float(l[2]), float(l[3])]
        return numpy.matrix([NE_count_dict[key] for key in keys])
    else:
        return numpy.matrix([NE_count(articles[u'data'][key][u'text']) for key in keys])

features = []
features_k = [named_entity]


def getX(key_list):
    x = vectorizer.transform(articles[u'data'][k][u'text'] for k in key_list)
    c_articles = [articles[u'data'][k] for k in key_list]
    for f in features :
        x_new = f(c_articles)
        x = scipy.sparse.hstack([x, x_new])
    for f in features_k :
        x_new = f(key_list)
        x = scipy.sparse.hstack([x, x_new])
    return x

In [ ]:
#[string.replace(string.replace(ca[u'byline'][u'original'], ' ', ''), '.', '') if ca[u'byline'] != [] else "" for ca in [articles[u'data'][k] for k in article_keys[:4000]]]

In [ ]:
X_train = getX(article_keys[:4000])
y_train = [k in T_keys for k in article_keys[:4000]]
X_dev = getX(article_keys[4000:])
y_dev = [k in T_keys for k in article_keys[4000:]]

In [ ]:
classifier = \
sklearn.linear_model.SGDClassifier(loss="log",
                                   penalty="elasticnet",
                                   n_iter=5)

_ = classifier.fit(X_train, y_train)

In [ ]:
pred = classifier.predict(X_dev)
print sklearn.metrics.accuracy_score(y_dev, pred)

In [ ]:
index_to_word = vectorizer.get_feature_names()
best_elts = numpy.argpartition(classifier.coef_, classifier.coef_.size - 20)[0][-20:]
best_words = [index_to_word[x] if x < len(index_to_word) else x-len(index_to_word) for x in best_elts]
best_words

In [ ]:
worst_elts = numpy.argpartition(classifier.coef_, 19)[0][:20]
worst_words = [index_to_word[x] if x < len(index_to_word) else x-len(index_to_word) for x in worst_elts]
worst_words

  (0, 1980)	0.0991455170763
  (0, 2189)	0.162397645228
  (0, 2197)	0.0516169289831
  (0, 2314)	0.0491153881479
  (0, 2729)	0.103925922652
  (0, 3105)	0.0307984579772
  (0, 3283)	0.067482745956
  (0, 3361)	0.0479323516441
  (0, 3632)	0.0875816080341
  (0, 3917)	0.175961882925
  (0, 3926)	0.103925922652
  (0, 4012)	0.0726357486844
  (0, 4015)	0.0248885676348
  (0, 4183)	0.0524104347367
  (0, 4838)	0.0439799587509
  (0, 5097)	0.0931229183095
  (0, 5129)	0.0382530477281
  (0, 5160)	0.0457913026157
  (0, 5531)	0.0422200321218
  (0, 5611)	0.065975599349
  (0, 5677)	0.0620086149554
  (0, 5849)	0.0454970644722
  (0, 5943)	0.0600002548772
  (0, 6448)	0.0667051060785
  (0, 6754)	0.0820399484338
  :	:
  (0, 66541)	0.0397347025829
  (0, 66602)	0.0554590041714
  (0, 66603)	0.0524852916919
  (0, 67275)	0.0259308659611
  (0, 67457)	0.0328956273242
  (0, 67506)	0.0289878530289
  (0, 68085)	0.133241693561
  (0, 68139)	0.0644307851994
  (0, 68165)	0.0426360022698
  (0, 68516)	0.0644307851994
  (0, 68572

In [271]:
#with open('NE_counts.txt', 'w') as f:
#    for i in range(4000):
#        f.write(article_keys[i] + ' ' + str(NE_train[i][0]) + ' ' + str(NE_train[i][1]) + ' ' + str(NE_train[i][2]) + '\n')
#    for i in range(len(NE_dev)):
#        f.write(article_keys[4000+i] + ' ' + str(NE_dev[i][0]) + ' ' + str(NE_dev[i][1]) + ' ' + str(NE_dev[i][2]) + '\n')

<4000x74928 sparse matrix of type '<type 'numpy.float64'>'
	with 1089537 stored elements in Compressed Sparse Row format>